In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [10]:
pseudo = pd.read_csv("../input/30-days-pseudo/submission.csv")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 26, 38),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 28, 40),
                'subsample': trial.suggest_uniform('subsample', 0.46, 0.66),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.01, 0.2),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 1.8e-2, 6.8e-2),
                 'min_child_samples':trial.suggest_int('min_child_samples', 47, 71),
              'num_leaves':trial.suggest_int('num_leaves', 48, 98),
              'max_depth':trial.suggest_int('max_depth', 2, 5),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 640, 840),
              'cat_smooth':trial.suggest_int('cat_smooth', 6, 26),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-4, 20)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-21 06:29:28,688] A new study created in memory with name: no-name-b8f57ccf-b305-4dca-a984-de529aeb03ab


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.721171
[2000]	valid_0's rmse: 0.716365
[3000]	valid_0's rmse: 0.713995
[4000]	valid_0's rmse: 0.712672
[5000]	valid_0's rmse: 0.71189
[6000]	valid_0's rmse: 0.711263
[7000]	valid_0's rmse: 0.710774
[8000]	valid_0's rmse: 0.710416
[9000]	valid_0's rmse: 0.710137
[10000]	valid_0's rmse: 0.709914
[11000]	valid_0's rmse: 0.70974
[12000]	valid_0's rmse: 0.7096
[13000]	valid_0's rmse: 0.709487
[14000]	valid_0's rmse: 0.709372
[15000]	valid_0's rmse: 0.709285
[16000]	valid_0's rmse: 0.709212
[17000]	valid_0's rmse: 0.709167
[18000]	valid_0's rmse: 0.709122
[19000]	valid_0's rmse: 0.709079
[20000]	valid_0's rmse: 0.709044
Did not meet early stopping. Best iteration is:
[19998]	valid_0's rmse: 0.709043
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.721338
[2000]	valid_0's rmse: 0.716672
[3000]	valid_0's rmse: 0.714354
[4000]	valid_0's rmse: 0.712941
[5000]	valid_0's rms

[I 2021-08-21 06:39:43,441] Trial 0 finished with value: 0.7089879305445157 and parameters: {'reg_lambda': 34.96966781554788, 'reg_alpha': 39.55932376216109, 'subsample': 0.6399276377000622, 'colsample_bytree': 0.09356517090010147, 'learning_rate': 0.02566081543332216, 'min_child_samples': 57, 'num_leaves': 63, 'max_depth': 3, 'max_bin': 834, 'cat_smooth': 9, 'cat_l2': 0.0009847396096553935}. Best is trial 0 with value: 0.7089879305445157.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.715773
[2000]	valid_0's rmse: 0.712157
[3000]	valid_0's rmse: 0.711118
[4000]	valid_0's rmse: 0.710682
[5000]	valid_0's rmse: 0.710472
[6000]	valid_0's rmse: 0.71038
[7000]	valid_0's rmse: 0.71033
[8000]	valid_0's rmse: 0.7103
[9000]	valid_0's rmse: 0.710284
[10000]	valid_0's rmse: 0.710274
[11000]	valid_0's rmse: 0.710265
[12000]	valid_0's rmse: 0.71026
[13000]	valid_0's rmse: 0.710257
[14000]	valid_0's rmse: 0.710252
[15000]	valid_0's rmse: 0.710251
Early stopping, best iteration is:
[15778]	valid_0's rmse: 0.710249
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.716019
[2000]	valid_0's rmse: 0.712303
[3000]	valid_0's rmse: 0.711206
[4000]	valid_0's rmse: 0.710752
[5000]	valid_0's rmse: 0.710568
[6000]	valid_0's rmse: 0.710485
[7000]	valid_0's rmse: 0.710431
[8000]	valid_0's rmse: 0.710399
[9000]	valid_0's rmse: 0.710382
[10000]	valid_0's rmse: 0.710371
[11000

[I 2021-08-21 06:44:45,005] Trial 1 finished with value: 0.7101958476715228 and parameters: {'reg_lambda': 29.30024551167937, 'reg_alpha': 36.75256251298534, 'subsample': 0.6591034559362348, 'colsample_bytree': 0.055207004203308395, 'learning_rate': 0.03685779067688042, 'min_child_samples': 49, 'num_leaves': 97, 'max_depth': 5, 'max_bin': 669, 'cat_smooth': 13, 'cat_l2': 3.9499394973313686}. Best is trial 0 with value: 0.7089879305445157.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.714904
[2000]	valid_0's rmse: 0.711793
[3000]	valid_0's rmse: 0.71052
[4000]	valid_0's rmse: 0.709932
[5000]	valid_0's rmse: 0.709595
[6000]	valid_0's rmse: 0.709407
[7000]	valid_0's rmse: 0.709287
Early stopping, best iteration is:
[7637]	valid_0's rmse: 0.709223
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.714963
[2000]	valid_0's rmse: 0.711837
[3000]	valid_0's rmse: 0.710539
[4000]	valid_0's rmse: 0.709874
[5000]	valid_0's rmse: 0.709538
[6000]	valid_0's rmse: 0.709352
[7000]	valid_0's rmse: 0.709219
[8000]	valid_0's rmse: 0.709152
[9000]	valid_0's rmse: 0.709115
[10000]	valid_0's rmse: 0.709084
Early stopping, best iteration is:
[10029]	valid_0's rmse: 0.709082


[I 2021-08-21 08:43:15,325] Trial 28 finished with value: 0.7090338454026993 and parameters: {'reg_lambda': 36.82425627383056, 'reg_alpha': 36.200869362060146, 'subsample': 0.5310885668347055, 'colsample_bytree': 0.19902569233001538, 'learning_rate': 0.04344414988914519, 'min_child_samples': 71, 'num_leaves': 85, 'max_depth': 3, 'max_bin': 647, 'cat_smooth': 19, 'cat_l2': 0.020958115711379826}. Best is trial 13 with value: 0.7086508785373714.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71533
[2000]	valid_0's rmse: 0.711959
[3000]	valid_0's rmse: 0.710586
[4000]	valid_0's rmse: 0.709896
[5000]	valid_0's rmse: 0.709584
[6000]	valid_0's rmse: 0.709319
[7000]	valid_0's rmse: 0.709163
[8000]	valid_0's rmse: 0.709102
[9000]	valid_0's rmse: 0.70902
[10000]	valid_0's rmse: 0.708978
[11000]	valid_0's rmse: 0.708941
[12000]	valid_0's rmse: 0.708914
Early stopping, best iteration is:
[12111]	valid_0's rmse: 0.708913
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71568
[2000]	valid_0's rmse: 0.712132
[3000]	valid_0's rmse: 0.710733
[4000]	valid_0's rmse: 0.710027
[5000]	valid_0's rmse: 0.709648
[6000]	valid_0's rmse: 0.70942
[7000]	valid_0's rmse: 0.709292
[8000]	valid_0's rmse: 0.709226
[9000]	valid_0's rmse: 0.70915
[10000]	valid_0's rmse: 0.709089
[11000]	valid_0's rmse: 0.709051
[12000]	valid_0's rmse: 0.709032
Early stopping, best iteration is:
[119

[I 2021-08-21 08:49:57,589] Trial 29 finished with value: 0.7087381931035919 and parameters: {'reg_lambda': 34.47669519680998, 'reg_alpha': 33.42624154799516, 'subsample': 0.4615745510813324, 'colsample_bytree': 0.10085909219048658, 'learning_rate': 0.039534745914564406, 'min_child_samples': 58, 'num_leaves': 68, 'max_depth': 4, 'max_bin': 755, 'cat_smooth': 12, 'cat_l2': 17.599967055620407}. Best is trial 13 with value: 0.7086508785373714.


Number of finished trials: 30
Best trial: {'reg_lambda': 35.530577151759196, 'reg_alpha': 38.00709309051285, 'subsample': 0.5919423807750431, 'colsample_bytree': 0.10128810432276648, 'learning_rate': 0.05340032565394751, 'min_child_samples': 60, 'num_leaves': 81, 'max_depth': 5, 'max_bin': 801, 'cat_smooth': 20, 'cat_l2': 18.747339614335626}


In [14]:
study.best_params

{'reg_lambda': 35.530577151759196,
 'reg_alpha': 38.00709309051285,
 'subsample': 0.5919423807750431,
 'colsample_bytree': 0.10128810432276648,
 'learning_rate': 0.05340032565394751,
 'min_child_samples': 60,
 'num_leaves': 81,
 'max_depth': 5,
 'max_bin': 801,
 'cat_smooth': 20,
 'cat_l2': 18.747339614335626}

# Log

====== Ordinal encoding =========

0.709167665584924 no noise ver9

0.7087162719562731 no noise ver16 (narrow space)

0.7127333982112984 row-wise noise ver10

0.7126600249836035 random noise ver11

====== One-hot encoding =========

0.7205030924594267 no noise ver13

0.722057702839371 random noise ver14

0.7222305639305339 row-wise noise ver15
